# 1. Descompactar arquivos:



In [ ]:
import zipfile
import os

zip_files = ["/content/vagas.zip", "/content/prospects.zip", "/content/applicants.zip"]

In [ ]:
temp_dir = "temp_files"
os.makedirs(temp_dir, exist_ok=True)

for zip_file in zip_files:
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(temp_dir)

#print(f"Files extracted to: {temp_dir}")

# 2. Ler os arquivos JSON

In [ ]:
json_files = []
for root, dirs, files in os.walk(temp_dir):
    for file in files:
        if file.endswith(".json"):
            json_files.append(os.path.join(root, file))

#print(json_files)

In [ ]:
import json
import pandas as pd

flat_dataframes = {}

for json_file_path in json_files:
    try:
        with open(json_file_path, 'r') as f:
            data = json.load(f)

            # Check if the data is a list of dictionaries
            if isinstance(data, list):
                # If it's a list, assume each element is a record and normalize
                df = pd.json_normalize(data)
            # Check if the data is a dictionary where values are dictionaries (like the 'vagas' structure)
            elif isinstance(data, dict) and all(isinstance(v, dict) for v in data.values()):
                 # Convert the dictionary of dictionaries to a list of dictionaries for normalization
                data_list = [{**{'id': k}, **v} for k, v in data.items()]
                df = pd.json_normalize(data_list)
            # Check if the data is a simple dictionary (like a single record)
            elif isinstance(data, dict):
                # If it's a single dictionary, normalize it directly
                df = pd.json_normalize(data)
            else:
                print(f"Skipping file '{json_file_path}' due to unsupported data structure.")
                continue

            dataframe_name = f"flat_{os.path.splitext(os.path.basename(json_file_path))[0]}"
            flat_dataframes[dataframe_name] = df
            print(f"DataFrame plano '{dataframe_name}' criado a partir de '{os.path.basename(json_file_path)}'.")

    except json.JSONDecodeError:
        print(f"Erro: O arquivo '{os.path.basename(json_file_path)}' não é um arquivo JSON válido.")
    except Exception as e:
        print(f"Ocorreu um erro ao processar o arquivo JSON '{os.path.basename(json_file_path)}': {e}")

# Now you can access your flat dataframes from the 'flat_dataframes' dictionary
# For example, to see the keys (flat dataframe names):
#print("\nFlat Dataframe Keys:")
#print(flat_dataframes.keys())

DataFrame plano 'flat_prospects' criado a partir de 'prospects.json'.
DataFrame plano 'flat_applicants' criado a partir de 'applicants.json'.
DataFrame plano 'flat_vagas' criado a partir de 'vagas.json'.


## Salvando os arquivos CSV, separado por ;

In [ ]:
# Create a directory to store the CSV files if it doesn't exist
os.makedirs(temp_dir, exist_ok=True)

# Iterate through the flat_dataframes dictionary and save each DataFrame as a CSV file
for name, df in flat_dataframes.items():
    csv_filename = os.path.join(temp_dir, f"{name}.csv")
    df.to_csv(csv_filename, index=False, sep=';')  # index=False to avoid writing the DataFrame index as a column
    #print(f"DataFrame '{name}' salvo como '{csv_filename}'.")

#print("\nTodos os DataFrames foram salvos como arquivos CSV.")

# 3. Tratamento dos dados

## Vagas

In [ ]:
flat_dataframes['flat_vagas']

,id,informacoes_basicas.data_requicisao,informacoes_basicas.limite_esperado_para_contratacao,informacoes_basicas.titulo_vaga,informacoes_basicas.vaga_sap,informacoes_basicas.cliente,informacoes_basicas.solicitante_cliente,informacoes_basicas.empresa_divisao,informacoes_basicas.requisitante,informacoes_basicas.analista_responsavel,...,perfil_vaga.demais_observacoes,perfil_vaga.viagens_requeridas,perfil_vaga.equipamentos_necessarios,beneficios.valor_venda,beneficios.valor_compra_1,beneficios.valor_compra_2,informacoes_basicas.data_inicial,informacoes_basicas.data_final,perfil_vaga.habilidades_comportamentais_necessarias,informacoes_basicas.nome_substituto
0,5185,04-05-2021,00-00-0000,Operation Lead -,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Srta. Bella Ferreira,...,100% Remoto Período – entre 5 – 6 meses,,Nenhum -,-,R$,,NaN,NaN,NaN,NaN
1,5184,04-05-2021,00-00-0000,Consultor PP/QM Sênior,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Yasmin da Rosa,...,• Início: Imediato • Fim: Jan/22,,Nenhum -,-,R$,,NaN,NaN,NaN,NaN
2,5183,04-05-2021,00-00-0000,ANALISTA PL/JR C/ SQL,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Ana Albuquerque,...,Localização: Remoto Perfil: Analista Pleno ou ...,,Nenhum -,-,R$,,NaN,NaN,NaN,NaN
3,5182,04-05-2021,18-05-2021,Technical Architect - 11894809,Não,Nelson-Page,Dr. Raul Monteiro,Decision São Paulo,Cecília Freitas,Clara Rios,...,Budgeted Rate - indicate currency and type (ho...,Não,Notebook padrão -,- p/ mês (168h),fechado,,18-05-2021,17-01-2022,NaN,NaN
4,5181,04-05-2021,00-00-0000,Consultor SAP AUTHORIZATION (BCA) -Pleno / Sênior,Não,Mann and Sons,Cauê Fogaça,Decision São Paulo,Maria Laura Nogueira,Srta. Bella Ferreira,...,contratação CLT full pela Decision locação rem...,Sim,Nenhum -,-,R$,,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14076,12368,28-02-2024,31-03-2024,PMP 5203658 - Dynamics,Não,Miller-Curry,Guilherme Campos,Decision Campinas,Sra. Juliana Oliveira,Sra. Joana Sousa,...,,,,"123,00 -",APPLICATION DEVELOPER-MICROSOFT ANALYTICS - ba...,,NaN,NaN,NaN,NaN
14077,12367,28-02-2024,29-02-2024,MM com IM - FSR 3910,Não,Bishop-Reed,Lívia Vasconcelos,Decision São Paulo,Manuella Jesus,Srta. Bella Ferreira,...,FSR: 3910,Não,,168 -,Fechado,,NaN,NaN,NaN,NaN
14078,12366,27-02-2024,29-02-2024,964 - Assistente fiscal ou financeiro,Não,"Glover, Warren and Norris",Lorenzo Cunha,Decision São Paulo,Maria Lopes,Maria Clara Pires,...,"Modelo de trabalho: Hibrido, 3 vezes por seman...",Não,,168 -,Fechado,,NaN,NaN,NaN,NaN
14079,12365,27-02-2024,29-02-2024,966 - Analista pleno ou sênior de área fiscal,Não,"Glover, Warren and Norris",Lorenzo Cunha,Decision São Paulo,Maria Lopes,Melina Montenegro,...,"Modelo de trabalho: Hibrido, 3 vezes por seman...",Não,,168 -,Fechado,,NaN,NaN,NaN,NaN


### Transformar ID em index

In [ ]:
if 'flat_vagas' in flat_dataframes:
    vagas_clear = flat_dataframes['flat_vagas'].copy()
if 'vagas_clear' in locals():
    vagas_clear = vagas_clear.set_index('id')
    print("Column 'id' set as index for DataFrame 'vagas_clear'.")
    #display(vagas_clear.head())
else:
    print("DataFrame 'vagas_clear' not found.")

Column 'id' set as index for DataFrame 'vagas_clear'.


### Eliminar colunas

In [ ]:
if 'vagas_clear' in locals():
    columns_to_drop = [
        'informacoes_basicas.empresa_divisao',
        'perfil_vaga.faixa_etaria',
        'informacoes_basicas.superior_imediato',
        'informacoes_basicas.nome',
        'informacoes_basicas.telefone',
        'perfil_vaga.pais',
        'perfil_vaga.bairro',
        'perfil_vaga.regiao',
        'perfil_vaga.local_trabalho',
        'beneficios.valor_venda',
        'beneficios.valor_compra_1',
        'beneficios.valor_compra_2',
        'informacoes_basicas.data_inicial',
        'informacoes_basicas.data_final',
        'informacoes_basicas.nome_substituto',
        'informacoes_basicas.cliente',
        'informacoes_basicas.solicitante_cliente',
        'informacoes_basicas.requisitante',
        'informacoes_basicas.analista_responsavel',
        'informacoes_basicas.origem_vaga',
        'perfil_vaga.cidade',
        'informacoes_basicas.data_requicisao',
        'informacoes_basicas.limite_esperado_para_contratacao',
        'informacoes_basicas.data_inicial',
        'informacoes_basicas.data_final'
    ]

    # Drop the specified columns if they exist in the DataFrame
    columns_to_drop_existing = [col for col in columns_to_drop if col in vagas_clear.columns]
    vagas_clear_filtered = vagas_clear.drop(columns=columns_to_drop_existing)

    print("Columns removed from 'vagas_clear'.")
    # Update the vagas_clear variable to the filtered DataFrame
    vagas_clear = vagas_clear_filtered
    #display(vagas_clear.head())
else:
    print("DataFrame 'vagas_clear' not found.")

Columns removed from 'vagas_clear'.


### Adicionar RR na coluna perfil_vaga.estado

In [ ]:
if 'vagas_clear' in locals():
    # Replace empty strings and potential NaN values with 'Roraima'
    vagas_clear['perfil_vaga.estado'] = vagas_clear['perfil_vaga.estado'].replace('', 'Roraima').fillna('Roraima')

    print("Empty cells in 'perfil_vaga.estado' column filled with 'Roraima'.")
    # Display the updated value counts for the column to verify the change
    print("\nUpdated value counts for 'perfil_vaga.estado':")
    #display(vagas_clear['perfil_vaga.estado'].value_counts(dropna=False))
else:
    print("DataFrame 'vagas_clear' not found.")

Empty cells in 'perfil_vaga.estado' column filled with 'Roraima'.

Updated value counts for 'perfil_vaga.estado':


### Converter em valores binários

In [ ]:
if 'vagas_clear' in locals():
    # Define the columns to convert to binary
    binary_columns = {
        'informacoes_basicas.vaga_sap': {'Não': 0, 'Sim': 1},
        'perfil_vaga.vaga_especifica_para_pcd': {'Não': 0, 'Sim': 1},
        'perfil_vaga.viagens_requeridas': {'Não': 0, 'Sim': 1},
        'informacoes_basicas.prazo_contratacao': {'Indeterminado': 0, 'Determinado': 1}
    }

    for col, mapping in binary_columns.items():
        if col in vagas_clear.columns:
            # Replace values based on the mapping. Use fillna(0) and replace('', 0)
            # to handle missing values and empty strings as 0.
            vagas_clear[col] = vagas_clear[col].replace(mapping).replace('', 0).fillna(0)
            print(f"Column '{col}' converted to binary.")
            # Display value counts to verify the conversion
            print(f"\nValue counts for '{col}':")
            #display(vagas_clear[col].value_counts(dropna=False))
        else:
            print(f"Column '{col}' not found in vagas_clear.")
else:
    print("DataFrame 'vagas_clear' not found.")

Column 'informacoes_basicas.vaga_sap' converted to binary.

Value counts for 'informacoes_basicas.vaga_sap':
Column 'perfil_vaga.vaga_especifica_para_pcd' converted to binary.

Value counts for 'perfil_vaga.vaga_especifica_para_pcd':
Column 'perfil_vaga.viagens_requeridas' converted to binary.

Value counts for 'perfil_vaga.viagens_requeridas':
Column 'informacoes_basicas.prazo_contratacao' converted to binary.

Value counts for 'informacoes_basicas.prazo_contratacao':


/tmp/ipython-input-11-1775791579.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  vagas_clear[col] = vagas_clear[col].replace(mapping).replace('', 0).fillna(0)


### Concatenar dados de texto

In [ ]:
if 'vagas_clear' in locals():
    columns_to_concatenate = {
        'informacoes_basicas.titulo_vaga': 'titulo vaga',
        'perfil_vaga.areas_atuacao': 'areas atuacao',
        'perfil_vaga.principais_atividades': 'principais atividades',
        'perfil_vaga.competencia_tecnicas_e_comportamentais': 'competencia tecnicas e comportamentais',
        'perfil_vaga.demais_observacoes': 'demais observacoes',
        'perfil_vaga.habilidades_comportamentais_necessarias': 'habilidades comportamentais necessarias'
    }

    concatenated_text = []
    original_columns_to_drop = []

    for col, prefix in columns_to_concatenate.items():
        if col in vagas_clear.columns:
            # Fill NaN values with empty strings and add the prefix
            prefixed_text = vagas_clear[col].fillna('').apply(lambda x: f"{prefix} {x}" if x else '')
            concatenated_text.append(prefixed_text)
            original_columns_to_drop.append(col)
        else:
            print(f"Column '{col}' not found in vagas_clear.")

    if concatenated_text:
        # Concatenate the prefixed text from all existing columns
        vagas_clear['informacoes.texto'] = pd.concat(concatenated_text, axis=1).agg(' '.join, axis=1)

        # Drop the original columns that were concatenated
        vagas_clear = vagas_clear.drop(columns=original_columns_to_drop)

        print("Text columns concatenated into 'informacoes.texto' with prefixes and original columns removed.")
        #display(vagas_clear.head())
    else:
        print("None of the specified text columns were found in 'vagas_clear' to concatenate.")
else:
    print("DataFrame 'vagas_clear' not found.")

Text columns concatenated into 'informacoes.texto' with prefixes and original columns removed.


### Dados Categóricos

#### Prioridade vaga

In [ ]:
if 'vagas_clear' in locals() and 'informacoes_basicas.prioridade_vaga' in vagas_clear.columns:
    # Example column
    vagas_clear['informacoes_basicas.prioridade_vaga'] = vagas_clear['informacoes_basicas.prioridade_vaga'].fillna('').str.strip()

    # Create auxiliary column with clean label
    vagas_clear['prioridade_label'] = vagas_clear['informacoes_basicas.prioridade_vaga'].apply(
        lambda x: (
            'Alta' if x.startswith('Alta')
            else 'Média' if x.startswith('Média')
            else 'Baixa' if x.startswith('Baixa')
            else 'Sem complexidade'
        )
    )

    # Map to numerical values
    prioridade_map = {
        'Alta': 3,
        'Média': 2,
        'Baixa': 1,
        'Sem complexidade': 0
    }

    vagas_clear['informacoes_basicas.prioridade_cat'] = vagas_clear['prioridade_label'].map(prioridade_map)

    # Drop the auxiliary and original columns
    columns_to_drop_prioridade = ['informacoes_basicas.prioridade_vaga', 'prioridade_label']
    vagas_clear = vagas_clear.drop(columns=columns_to_drop_prioridade, errors='ignore') # Use errors='ignore' to avoid errors if columns are already dropped

    #display(vagas_clear[['informacoes_basicas.prioridade_cat']].head()) # Display only the new numerical column
else:
    print("DataFrame 'vagas_clear' or the column 'informacoes_basicas.prioridade_vaga' not found.")

#### Objetivo da Vaga

In [ ]:
if 'vagas_clear' in locals() and 'informacoes_basicas.objetivo_vaga' in vagas_clear.columns:
    # Perform one-hot encoding
    objetivo_vaga_one_hot = pd.get_dummies(vagas_clear['informacoes_basicas.objetivo_vaga'], prefix='objetivo_vaga')

    # Concatenate the one-hot encoded columns to the original DataFrame
    vagas_clear = pd.concat([vagas_clear, objetivo_vaga_one_hot], axis=1)

    # Drop the original 'informacoes_basicas.objetivo_vaga' column
    vagas_clear = vagas_clear.drop(columns=['informacoes_basicas.objetivo_vaga'])

    print("One-hot encoding applied to 'informacoes_basicas.objetivo_vaga' and original column dropped.")
    display(vagas_clear.head())

else:
    print("DataFrame 'vagas_clear' or the column 'informacoes_basicas.objetivo_vaga' not found.")

One-hot encoding applied to 'informacoes_basicas.objetivo_vaga' and original column dropped.


,informacoes_basicas.vaga_sap,informacoes_basicas.tipo_contratacao,informacoes_basicas.prazo_contratacao,perfil_vaga.estado,perfil_vaga.vaga_especifica_para_pcd,perfil_vaga.horario_trabalho,perfil_vaga.nivel profissional,perfil_vaga.nivel_academico,perfil_vaga.nivel_ingles,perfil_vaga.nivel_espanhol,...,perfil_vaga.viagens_requeridas,perfil_vaga.equipamentos_necessarios,informacoes.texto,informacoes_basicas.prioridade_cat,objetivo_vaga_,objetivo_vaga_Contratação,objetivo_vaga_Ordem de Serviço,objetivo_vaga_Parcerias,objetivo_vaga_Prospecção,objetivo_vaga_RFP
id,,,,,,,,,,,,,,,,,,,,,
5185,0,CLT Full,0,São Paulo,0,,Sênior,Ensino Superior Completo,Avançado,Fluente,...,0,Nenhum -,titulo vaga Operation Lead - areas atuacao TI ...,0,True,False,False,False,False,False
5184,0,CLT Full,0,São Paulo,0,,Sênior,Ensino Superior Completo,Fluente,Nenhum,...,0,Nenhum -,titulo vaga Consultor PP/QM Sênior areas atuac...,0,False,True,False,False,False,False
5183,0,CLT Full,0,São Paulo,0,,Analista,Ensino Superior Completo,Nenhum,Intermediário,...,0,Nenhum -,titulo vaga ANALISTA PL/JR C/ SQL areas atuaca...,0,False,False,False,False,False,True
5182,0,"PJ/Autônomo, CLT Full",1,São Paulo,0,,Analista,Ensino Superior Completo,Básico,Básico,...,0,Notebook padrão -,titulo vaga Technical Architect - 11894809 are...,3,False,True,False,False,False,False
5181,0,CLT Full,0,São Paulo,0,,Sênior,Ensino Superior Completo,Intermediário,Nenhum,...,1,Nenhum -,titulo vaga Consultor SAP AUTHORIZATION (BCA) ...,0,True,False,False,False,False,False


#### Horario de Trabalho

In [ ]:
faixa_horaria_normalizada = {
    # Horário Comercial (
    "09:00 as 18:00": "Comercial",
    "9/18": "Comercial",
    "Comercial": "Comercial",
    "09 horas as 18 horas": "Comercial",
    "9 as 18": "Comercial",
    "Horario comercial": "Comercial",
    "Comercial - 09h às 18h segunda a sexta": "Comercial",
    "9 às 18 hs": "Comercial",
    "Das 9 às 18hs": "Comercial",
    "9 AS 18": "Comercial",
    "das *9 as 18hs": "Comercial",
    "9h as 18h": "Comercial",
    "Horário Comercial": "Comercial",
    "DAS 09:00 AS 18:00": "Comercial",

    # Variações parecidas
    "08:30 às 12:30 hrs / 14:00 às 18:00 hrs": "08:00–18:00",
    "8 a 17": "08:00–17:00",
    "8 as 18": "08:00–18:00",
    "8:00 às 17:00": "08:00–17:00",
    "08:00 as 17:00": "08:00–17:00",
    "8:00am to 5:00pm": "08:00–17:00",
    "8/17": "08:00–17:00",
    "Horário Comercial: das 8 às 18 horas": "08:00–18:00",
    "09h as 187h": "08:00–18:00",

    # Outros horários específicos
    "Das 11h às 20h": "11:00–20:00",
    "de segunda a domingo, das 22:00 às 06:00 horas com": "22:00–06:00",

    # Casos não estruturados ou irrelevantes como local
    "Vide descrição da vaga": "Indefinido",
    "hibrido 3 x por semana na Franciso Matarazzo proxi": "Indefinido",
    "Atuação em São Bernardo do Campo - entre Pampas e": "Indefinido",
    "Vaga para cobertura de licença maternidade, em tor": "Indefinido",

    # Turnos alternativos
    "6x1 / madrugada 5x2": "Turno variável",
    "02 vagas - Das 09h às 18h e das 10 às 19h": "Turno variável",
    "9/18 14/22": "Turno variável"
}


In [ ]:
if 'vagas_clear' in locals() and 'perfil_vaga.horario_trabalho' in vagas_clear.columns and 'faixa_horaria_normalizada' in locals():
    # Apply the normalization dictionary
    vagas_clear['horario_trabalho_normalizado'] = vagas_clear['perfil_vaga.horario_trabalho'].map(faixa_horaria_normalizada).fillna('Indefinido')

    # Perform one-hot encoding on the normalized column
    horario_trabalho_one_hot = pd.get_dummies(vagas_clear['horario_trabalho_normalizado'], prefix='horario_trabalho')

    # Concatenate the one-hot encoded columns to the original DataFrame
    vagas_clear = pd.concat([vagas_clear, horario_trabalho_one_hot], axis=1)

    # Drop the original and normalized columns
    vagas_clear = vagas_clear.drop(columns=['perfil_vaga.horario_trabalho', 'horario_trabalho_normalizado'])

    print("Normalization and one-hot encoding applied to 'perfil_vaga.horario_trabalho'. Original columns dropped.")
    #display(vagas_clear.head())
else:
    print("DataFrame 'vagas_clear', column 'perfil_vaga.horario_trabalho', or dictionary 'faixa_horaria_normalizada' not found.")

Normalization and one-hot encoding applied to 'perfil_vaga.horario_trabalho'. Original columns dropped.


#### Tipo de Contratação

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

if 'vagas_clear' in locals() and 'informacoes_basicas.tipo_contratacao' in vagas_clear.columns:
    # Corrigir nulos ou vazios
    vagas_clear['informacoes_basicas.tipo_contratacao'] = vagas_clear['informacoes_basicas.tipo_contratacao'].fillna('').str.strip()

    # Separar os tipos por vírgula e remover espaços extras
    vagas_clear['tipos_lista'] = vagas_clear['informacoes_basicas.tipo_contratacao'].apply(
        lambda x: [tipo.strip() for tipo in x.split(',') if tipo.strip()]
    )

    # Obter todos os tipos únicos
    mlb = MultiLabelBinarizer()
    df_tipos = pd.DataFrame(mlb.fit_transform(vagas_clear['tipos_lista']), columns=[f"tipo_{t}" for t in mlb.classes_], index=vagas_clear.index)

    # Juntar ao DataFrame original
    vagas_clear = pd.concat([vagas_clear, df_tipos], axis=1)

    # (Opcional) Contar quantos tipos estão listados por linha
    vagas_clear['num_tipos_contratacao'] = vagas_clear['tipos_lista'].apply(len)

    # Se quiser, pode agora remover colunas auxiliares:
    vagas_clear.drop(columns=['informacoes_basicas.tipo_contratacao', 'tipos_lista'], inplace=True)

    print("Multi-label binarization applied to 'informacoes_basicas.tipo_contratacao'.")
    display(vagas_clear.head())

else:
    print("DataFrame 'vagas_clear' or the column 'informacoes_basicas.tipo_contratacao' not found.")

Multi-label binarization applied to 'informacoes_basicas.tipo_contratacao'.


,informacoes_basicas.vaga_sap,informacoes_basicas.prazo_contratacao,perfil_vaga.estado,perfil_vaga.vaga_especifica_para_pcd,perfil_vaga.nivel profissional,perfil_vaga.nivel_academico,perfil_vaga.nivel_ingles,perfil_vaga.nivel_espanhol,perfil_vaga.outro_idioma,perfil_vaga.viagens_requeridas,...,horario_trabalho_Indefinido,horario_trabalho_Turno variável,tipo_CLT Cotas,tipo_CLT Full,tipo_Candidato poderá escolher,tipo_Cooperado,tipo_Estagiário,tipo_Hunting,tipo_PJ/Autônomo,num_tipos_contratacao
id,,,,,,,,,,,,,,,,,,,,,
5185,0,0,São Paulo,0,Sênior,Ensino Superior Completo,Avançado,Fluente,,0,...,True,False,0,1,0,0,0,0,0,1
5184,0,0,São Paulo,0,Sênior,Ensino Superior Completo,Fluente,Nenhum,,0,...,True,False,0,1,0,0,0,0,0,1
5183,0,0,São Paulo,0,Analista,Ensino Superior Completo,Nenhum,Intermediário,,0,...,True,False,0,1,0,0,0,0,0,1
5182,0,1,São Paulo,0,Analista,Ensino Superior Completo,Básico,Básico,,0,...,True,False,0,1,0,0,0,0,1,2
5181,0,0,São Paulo,0,Sênior,Ensino Superior Completo,Intermediário,Nenhum,,1,...,True,False,0,1,0,0,0,0,0,1


#### Estado

In [ ]:
if 'vagas_clear' in locals() and 'perfil_vaga.estado' in vagas_clear.columns:
    # Perform one-hot encoding
    estado_one_hot = pd.get_dummies(vagas_clear['perfil_vaga.estado'], prefix='estado')

    # Concatenate the one-hot encoded columns to the original DataFrame
    vagas_clear = pd.concat([vagas_clear, estado_one_hot], axis=1)

    # Drop the original 'perfil_vaga.estado' column
    vagas_clear = vagas_clear.drop(columns=['perfil_vaga.estado'])

    print("One-hot encoding applied to 'perfil_vaga.estado' and original column dropped.")
    #display(vagas_clear.head())

else:
    print("DataFrame 'vagas_clear' or the column 'perfil_vaga.estado' not found.")

One-hot encoding applied to 'perfil_vaga.estado' and original column dropped.


#### Equipamento

In [ ]:
if 'vagas_clear' in locals() and 'perfil_vaga.equipamentos_necessarios' in vagas_clear.columns:
    # Garantir string e remover espaços
    vagas_clear['perfil_vaga.equipamentos_necessarios'] = vagas_clear['perfil_vaga.equipamentos_necessarios'].fillna('').str.strip()

    # Nova função
    def precisa_equipamento(valor):
        if valor == '':
            return 0
        itens = [item.strip().lower() for item in valor.split('-') if item.strip()]
        if any('nenhum' in item for item in itens):
            return 0
        return 1

    # Aplicar
    vagas_clear['precisa_equipamento'] = vagas_clear['perfil_vaga.equipamentos_necessarios'].apply(precisa_equipamento)

    # Drop the original column
    vagas_clear = vagas_clear.drop(columns=['perfil_vaga.equipamentos_necessarios'])

    print("Column 'perfil_vaga.equipamentos_necessarios' processed to create 'precisa_equipamento' binary column and original column dropped.")
    # Optional: Display value counts for the new column
    # print("\nValue counts for 'precisa_equipamento':")
    # display(vagas_clear['precisa_equipamento'].value_counts(dropna=False))

else:
    print("DataFrame 'vagas_clear' or the column 'perfil_vaga.equipamentos_necessarios' not found.")

Column 'perfil_vaga.equipamentos_necessarios' processed to create 'precisa_equipamento' binary column and original column dropped.


#### Nível

In [ ]:
# Dicionário de mapeamento
nivel_map = {
    'Aprendiz': 0,
    'Auxiliar': 1,
    'Assistente': 2,
    'Técnico de Nível Médio': 3,
    'Júnior': 4,
    'Pleno': 5,
    'Analista': 6,
    'Sênior': 7,
    'Especialista': 8,
    'Líder': 9,
    'Supervisor': 10,
    'Coordenador': 11,
    'Gerente': 12,
    'Trainee': 0
}

if 'vagas_clear' in locals() and 'perfil_vaga.nivel profissional' in vagas_clear.columns:
    # Aplicar o mapeamento
    vagas_clear['nivel_profissional_ordinal'] = vagas_clear['perfil_vaga.nivel profissional'].map(nivel_map)

    # Drop the original column
    vagas_clear = vagas_clear.drop(columns=['perfil_vaga.nivel profissional'])

    print("Column 'perfil_vaga.nivel profissional' mapped to 'nivel_profissional_ordinal' and original column dropped.")
    # Optional: Display value counts for the new column
    # print("\nValue counts for 'nivel_profissional_ordinal':")
    # display(vagas_clear['nivel_profissional_ordinal'].value_counts(dropna=False))
else:
    print("DataFrame 'vagas_clear' or the column 'perfil_vaga.nivel profissional' not found.")

Column 'perfil_vaga.nivel profissional' mapped to 'nivel_profissional_ordinal' and original column dropped.


In [ ]:
nivel_academico_map = {
    'Ensino Fundamental Completo': 1,
    'Ensino Médio Incompleto': 2,
    'Ensino Médio Completo': 3,
    'Ensino Técnico Incompleto': 4,
    'Ensino Técnico Cursando': 5,
    'Ensino Técnico Completo': 6,
    'Ensino Superior Incompleto': 7,
    'Ensino Superior Cursando': 8,
    'Ensino Superior Completo': 9,
    'Pós Graduação Incompleto': 10,
    'Pós Graduação Cursando': 11,
    'Pós Graduação Completo': 12,
    'Mestrado Cursando': 13,
    'Mestrado Completo': 14,
    'Doutorado Cursando': 15,
    'Doutorado Completo': 16
}

if 'vagas_clear' in locals() and 'perfil_vaga.nivel_academico' in vagas_clear.columns:
    vagas_clear['nivel_academico_ordinal'] = vagas_clear['perfil_vaga.nivel_academico'].map(nivel_academico_map)
    # Drop the original column
    vagas_clear = vagas_clear.drop(columns=['perfil_vaga.nivel_academico'])
    print("Column 'perfil_vaga.nivel_academico' mapped to 'nivel_academico_ordinal' and original column dropped.")
    # Optional: Display value counts for the new column
    # print("\nValue counts for 'nivel_academico_ordinal':")
    # display(vagas_clear['nivel_academico_ordinal'].value_counts(dropna=False))
else:
    print("DataFrame 'vagas_clear' or the column 'perfil_vaga.nivel_academico' not found.")

Column 'perfil_vaga.nivel_academico' mapped to 'nivel_academico_ordinal' and original column dropped.


In [ ]:
nivel_ingles_map = {
    'Nenhum': 0,
    'Técnico': 1,
    'Básico': 2,
    'Intermediário': 3,
    'Avançado': 4,
    'Fluente': 5
}

if 'vagas_clear' in locals() and 'perfil_vaga.nivel_ingles' in vagas_clear.columns:
    vagas_clear['nivel_ingles_ordinal'] = vagas_clear['perfil_vaga.nivel_ingles'].map(nivel_ingles_map)
    # Drop the original column
    vagas_clear = vagas_clear.drop(columns=['perfil_vaga.nivel_ingles'])
    print("Column 'perfil_vaga.nivel_ingles' mapped to 'nivel_ingles_ordinal' and original column dropped.")
    # Optional: Display value counts for the new column
    # print("\nValue counts for 'nivel_ingles_ordinal':")
    # display(vagas_clear['nivel_ingles_ordinal'].value_counts(dropna=False))
else:
    print("DataFrame 'vagas_clear' or the column 'perfil_vaga.nivel_ingles' not found.")

Column 'perfil_vaga.nivel_ingles' mapped to 'nivel_ingles_ordinal' and original column dropped.


In [ ]:
# Substituir valores vazios por 'Nenhum' e padronizar
col = 'perfil_vaga.nivel_espanhol'
if 'vagas_clear' in locals() and col in vagas_clear.columns:
    vagas_clear[col] = vagas_clear[col].fillna('').str.strip()
    vagas_clear[col] = vagas_clear[col].replace('', 'Nenhum')

    # Mapeamento ordinal
    nivel_espanhol_map = {
        'Nenhum': 0,
        'Técnico': 1,
        'Básico': 2,
        'Intermediário': 3,
        'Avançado': 4,
        'Fluente': 5
    }

    # Aplicar
    vagas_clear['nivel_espanhol_ordinal'] = vagas_clear[col].map(nivel_espanhol_map)

    # Drop the original column
    vagas_clear = vagas_clear.drop(columns=[col])

    print(f"Column '{col}' mapped to 'nivel_espanhol_ordinal' and original column dropped.")
    # Optional: Display value counts for the new column
    # print(f"\nValue counts for 'nivel_espanhol_ordinal':")
    # display(vagas_clear['nivel_espanhol_ordinal'].value_counts(dropna=False))
else:
    print(f"DataFrame 'vagas_clear' or the column '{col}' not found.")

Column 'perfil_vaga.nivel_espanhol' mapped to 'nivel_espanhol_ordinal' and original column dropped.


In [ ]:
col = 'perfil_vaga.outro_idioma'
if 'vagas_clear' in locals() and col in vagas_clear.columns:
    # Preparar a coluna
    vagas_clear[col] = vagas_clear[col].fillna('').str.strip()

    def tem_idioma_adicional(valor):
        valor_lower = valor.lower()
        if (
            valor_lower in ['', 'nenhum'] or
            ('português' in valor_lower and all(lang in valor_lower for lang in ['português']) and not any(lang in valor_lower for lang in ['francês', 'mandarim', 'alemão', 'russo', 'inglês']))
        ):
            return 0
        return 1

    vagas_clear['tem_outro_idioma'] = vagas_clear[col].apply(tem_idioma_adicional)

    # Drop the original column
    vagas_clear = vagas_clear.drop(columns=[col])

    print(f"Column '{col}' processed to create 'tem_outro_idioma' binary column and original column dropped.")
    # Optional: Display value counts for the new column
    # print(f"\nValue counts for 'tem_outro_idioma':")
    # display(vagas_clear['tem_outro_idioma'].value_counts(dropna=False))
else:
    print(f"DataFrame 'vagas_clear' or the column '{col}' not found.")

Column 'perfil_vaga.outro_idioma' processed to create 'tem_outro_idioma' binary column and original column dropped.


### Opcional: Salvar CSV

In [ ]:
import os

# Create a directory to store the CSV files if it doesn't exist
output_dir = "temp_files"
os.makedirs(output_dir, exist_ok=True)

# Save the vagas_clear DataFrame as a CSV file
if 'vagas_clear' in locals():
    csv_filename = os.path.join(output_dir, "vagas_clear.csv")
    vagas_clear.to_csv(csv_filename, index=True, sep=';') # index=True to keep the 'id' as index in the CSV
    print(f"DataFrame 'vagas_clear' salvo como '{csv_filename}'.")
else:
    print("DataFrame 'vagas_clear' não encontrado.")

DataFrame 'vagas_clear' salvo como 'temp_files/vagas_clear.csv'.


## Applicants

### Transformar ID em index

In [ ]:
if 'flat_applicants' in flat_dataframes:
    applicants_clear = flat_dataframes['flat_applicants'].copy() # Create a copy to avoid modifying the original
    if 'id' in applicants_clear.columns:
        applicants_clear = applicants_clear.set_index('id')
        print("Column 'id' set as index for DataFrame 'applicants_clear'.")
        #display(applicants_clear.head())
    else:
        print("Column 'id' not found in DataFrame 'flat_applicants'.")
else:
    print("DataFrame 'flat_applicants' not found.")

Column 'id' set as index for DataFrame 'applicants_clear'.


### Eliminar colunas

In [ ]:
if 'applicants_clear' in locals():
    columns_to_drop = [
        'cv_en',
        'infos_basicas.telefone_recado',
        'infos_basicas.sabendo_de_nos_por',
        'informacoes_pessoais.data_aceite',
        'informacoes_pessoais.cpf',
        'informacoes_pessoais.fonte_indicacao',
        'informacoes_pessoais.email_secundario',
        'informacoes_pessoais.data_nascimento',
        'informacoes_pessoais.telefone_recado',
        'informacoes_pessoais.sexo',
        'informacoes_pessoais.estado_civil',
        'informacoes_pessoais.skype',
        'informacoes_pessoais.url_linkedin',
        'informacoes_pessoais.facebook',
        'informacoes_pessoais.download_cv',
        'cargo_atual.id_ibrati',
        'infos_basicas.data_criacao',
        'cargo_atual.email_corporativo',
        'cargo_atual.projeto_atual',
        'infos_basicas.codigo_profissional',
        'infos_basicas.data_atualizacao',
        'infos_basicas.nome',
        'cargo_atual.unidade',
        'infos_basicas.email',
        'infos_basicas.inserido_por',
        'cargo_atual.nome_superior_imediato',
        'cargo_atual.email_superior_imediato',
        'informacoes_pessoais.nome',
        'informacoes_pessoais.email',
        'informacoes_profissionais.remuneracao',
        'formacao_e_idiomas.ano_conclusao',
        'cargo_atual.data_admissao',
        'cargo_atual.data_ultima_promocao',
        'formacao_e_idiomas.instituicao_ensino_superior',
        'cargo_atual.cliente'
    ]

    # Drop the specified columns if they exist in the DataFrame
    columns_to_drop_existing = [col for col in columns_to_drop if col in applicants_clear.columns]
    applicants_clear_filtered = applicants_clear.drop(columns=columns_to_drop_existing)

    print("Columns removed from 'applicants_clear'.")
    # Update the applicants_clear variable to the filtered DataFrame
    applicants_clear = applicants_clear_filtered
    #display(applicants_clear.head())
else:
    print("DataFrame 'applicants_clear' not found.")

Columns removed from 'applicants_clear'.


### Concatenar dados de texto

In [ ]:
if 'applicants_clear' in locals():
    columns_to_concatenate = {
        'cv_pt': 'cv pt',
        'infos_basicas.objetivo_profissional': 'objetivo profissional',
        'informacoes_profissionais.titulo_profissional': 'titulo profissional',
        'informacoes_profissionais.area_atuacao': 'area atuacao',
        'informacoes_profissionais.conhecimentos_tecnicos': 'conhecimentos tecnicos',
        'informacoes_profissionais.certificacoes': 'certificacoes',
        'informacoes_profissionais.outras_certificacoes': 'outras certificacoes',
        'formacao_e_idiomas.cursos': 'cursos',
        'informacoes_profissionais.qualificacoes': 'qualificacoes',
        'formacao_e_idiomas.outro_curso': 'outro curso',
        'cargo_atual.cargo_atual': 'cargo atual',
        'informacoes_profissionais.experiencias': 'experiencias'
    }

    concatenated_text = []
    original_columns_to_drop = []

    for col, prefix in columns_to_concatenate.items():
        if col in applicants_clear.columns:
            # Fill NaN values with empty strings and add the prefix
            prefixed_text = applicants_clear[col].fillna('').apply(lambda x: f"{prefix} {x}" if x else '')
            concatenated_text.append(prefixed_text)
            original_columns_to_drop.append(col)
        else:
            print(f"Column '{col}' not found in applicants_clear.")

    if concatenated_text:
        # Concatenate the prefixed text from all existing columns
        applicants_clear['informacoes.texto'] = pd.concat(concatenated_text, axis=1).agg(' '.join, axis=1)

        # Drop the original columns that were concatenated
        applicants_clear = applicants_clear.drop(columns=original_columns_to_drop)

        print("Text columns concatenated into 'informacoes.texto' with prefixes and original columns removed from 'applicants_clear'.")
        #display(applicants_clear.head())
    else:
        print("None of the specified text columns were found in 'applicants_clear' to concatenate.")
else:
    print("DataFrame 'applicants_clear' not found.")

Text columns concatenated into 'informacoes.texto' with prefixes and original columns removed from 'applicants_clear'.


### Dados Binários

In [ ]:
if 'applicants_clear' in locals():
    # Define the columns to convert to binary
    binary_columns = {
        'informacoes_pessoais.pcd': {'Não': 0, 'Sim': 1},
        }

    for col, mapping in binary_columns.items():
        if col in applicants_clear.columns:
            # Replace values based on the mapping. Use fillna(0) and replace('', 0)
            # to handle missing values and empty strings as 0.
            applicants_clear[col] = applicants_clear[col].replace(mapping).replace('', 0).fillna(0)
            print(f"Column '{col}' converted to binary.")
            # Display value counts to verify the conversion
            print(f"\nValue counts for '{col}':")
            #display(applicants_clear[col].value_counts(dropna=False))
        else:
            print(f"Column '{col}' not found in applicants_clear.")
else:
    print("DataFrame 'applicants_clear' not found.")

Column 'informacoes_pessoais.pcd' converted to binary.

Value counts for 'informacoes_pessoais.pcd':


/tmp/ipython-input-30-2692133523.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  applicants_clear[col] = applicants_clear[col].replace(mapping).replace('', 0).fillna(0)


In [ ]:
col = 'formacao_e_idiomas.outro_idioma'

if 'applicants_clear' in locals() and col in applicants_clear.columns:
    # Preparar a coluna
    applicants_clear[col] = applicants_clear[col].fillna('').str.strip()

    # Lista de idiomas que NÃO são considerados "outros"
    idiomas_nativos = ['português']

    def tem_idioma_adicional(valor):
        valor_lower = valor.lower()

        # Caso o campo esteja vazio ou indique "nenhum"
        if valor_lower in ['', '-', 'nenhum', 'português - nenhum']:
            return 0

        # Se contiver apenas português (em qualquer nível)
        idiomas_mencionados = [i.split('-')[0].strip() for i in valor_lower.split('\n') if i.strip()]
        idiomas_mencionados = list(set(idiomas_mencionados))  # remover duplicatas

        outros_idiomas = [i for i in idiomas_mencionados if i not in idiomas_nativos]

        if outros_idiomas:
            return 1
        else:
            return 0

    # Aplica a função e cria a nova coluna binária
    applicants_clear['tem_outro_idioma'] = applicants_clear[col].apply(tem_idioma_adicional)

    # Remove a coluna original
    applicants_clear = applicants_clear.drop(columns=[col])

    print(f"Coluna '{col}' processada e 'tem_outro_idioma' criada com sucesso.")
else:
    print(f"DataFrame 'applicants_clear' ou a coluna '{col}' não encontrada.")


Coluna 'formacao_e_idiomas.outro_idioma' processada e 'tem_outro_idioma' criada com sucesso.


### Estado

In [ ]:
import re

ddd_estado = {
    "11": "São Paulo", "12": "São Paulo", "13": "São Paulo", "14": "São Paulo", "15": "São Paulo", "16": "São Paulo", "17": "São Paulo", "18": "São Paulo", "19": "São Paulo",
    "21": "Rio de Janeiro", "22": "Rio de Janeiro", "24": "Rio de Janeiro",
    "27": "Espírito Santo", "28": "Espírito Santo",
    "31": "Minas Gerais", "32": "Minas Gerais", "33": "Minas Gerais", "34": "Minas Gerais", "35": "Minas Gerais", "37": "Minas Gerais", "38": "Minas Gerais",
    "41": "Paraná", "42": "Paraná", "43": "Paraná", "44": "Paraná", "45": "Paraná", "46": "Paraná",
    "47": "Santa Catarina", "48": "Santa Catarina", "49": "Santa Catarina",
    "51": "Rio Grande do Sul", "53": "Rio Grande do Sul", "54": "Rio Grande do Sul", "55": "Rio Grande do Sul",
    "61": "Distrito Federal",
    "62": "Goiás", "64": "Goiás",
    "63": "Tocantins",
    "65": "Mato Grosso", "66": "Mato Grosso",
    "67": "Mato Grosso do Sul",
    "68": "Acre",
    "69": "Rondônia",
    "71": "Bahia", "73": "Bahia", "74": "Bahia", "75": "Bahia", "77": "Bahia",
    "79": "Sergipe",
    "81": "Pernambuco", "87": "Pernambuco",
    "82": "Alagoas",
    "83": "Paraíba",
    "84": "Rio Grande do Norte",
    "85": "Ceará", "88": "Ceará",
    "86": "Piauí", "89": "Piauí",
    "91": "Pará", "93": "Pará", "94": "Pará",
    "92": "Amazonas", "97": "Amazonas",
    "95": "Roraima",
    "96": "Amapá",
    "98": "Maranhão", "99": "Maranhão"
}

# Mapping from full state name (lowercase) to abbreviation (not needed for return value, but useful for lookup)
estado_nome_para_sigla = {
    "são paulo": "SP", "rio de janeiro": "RJ", "espírito santo": "ES",
    "minas gerais": "MG", "paraná": "PR", "santa catarina": "SC",
    "rio grande do sul": "RS", "distrito federal": "DF", "goiás": "GO",
    "tocantins": "TO", "mato grosso": "MT", "mato grosso do sul": "MS",
    "acre": "AC", "rondônia": "RO", "bahia": "BA", "sergipe": "SE",
    "pernambuco": "PE", "alagoas": "AL", "paraíba": "PB", "rio grande do norte": "RN",
    "ceará": "CE", "piauí": "PI", "pará": "PA", "amazonas": "AM", "roraima": "RR",
    "amapá": "AP", "maranhão": "MA"
}

# Reverse mapping from abbreviation to full state name (for return value)
estado_sigla_para_nome = {v: k for k, v in estado_nome_para_sigla.items()}


def extrair_estado(row):
    # 1. informacoes_pessoais.endereco -> estado em minúsculo (nome completo)
    endereco = str(row.get('informacoes_pessoais.endereco', '')).lower()
    # Regex to find a potential state name at the end of a phrase, after a comma
    match = re.search(r',\s*([a-z\s]+)$', endereco)
    if match:
        estado_encontrado = match.group(1).strip()
        # Check if the extracted part is a full state name (lowercase) and return capitalized
        if estado_encontrado in estado_nome_para_sigla:
            # Find the corresponding full name in ddd_estado and capitalize it
            for sigla, nome_completo in ddd_estado.items():
                if nome_completo.lower() == estado_encontrado:
                    return nome_completo # Return the capitalized version from ddd_estado

    # 2. Local no formato 'Cidade, Estado' (Estado em maiúscula por extenso)
    local = str(row.get('infos_basicas.local', ''))
    if ',' in local:
        partes = local.split(',')
        if len(partes) > 1:
            estado_parte = partes[1].strip()
            # Check if it's a full state name (case-sensitive for uppercase) and return as is
            if estado_parte in ddd_estado.values():
                return estado_parte

    # 3. DDD do infos_basicas.telefone
    tel1 = str(row.get('infos_basicas.telefone', ''))
    ddd1 = re.findall(r'\(?(\d{2})\)?', tel1)
    if ddd1 and ddd1[0] in ddd_estado:
        return ddd_estado[ddd1[0]] # Return the capitalized full name from ddd_estado

    # 4. DDD do informacoes_pessoais.telefone_celular
    tel2 = str(row.get('informacoes_pessoais.telefone_celular', ''))
    ddd2 = re.findall(r'\(?(\d{2})\)?', tel2)
    if ddd2 and ddd2[0] in ddd_estado:
        return ddd_estado[ddd2[0]] # Return the capitalized full name from ddd_estado

    return None  # Nenhuma informação válida encontrada

In [ ]:
applicants_clear['estado'] = applicants_clear.apply(extrair_estado, axis=1)
applicants_clear = pd.get_dummies(applicants_clear, columns=['estado'], prefix='estado')
applicants_clear = applicants_clear.drop(columns=['informacoes_pessoais.endereco', 'infos_basicas.local', 'infos_basicas.telefone', 'informacoes_pessoais.telefone_celular'])

### Nível

In [ ]:
# Dicionário de mapeamento
nivel_map = {
    'Aprendiz': 0,
    'Auxiliar': 1,
    'Assistente': 2,
    'Técnico de Nível Médio': 3,
    'Júnior': 4,
    'Pleno': 5,
    'Analista': 6,
    'Sênior': 7,
    'Especialista': 8,
    'Líder': 9,
    'Supervisor': 10,
    'Coordenador': 11,
    'Gerente': 12,
    'Trainee': 0
}

if 'applicants_clear' in locals() and 'informacoes_profissionais.nivel_profissional' in applicants_clear.columns:
    # Aplicar o mapeamento
    applicants_clear['nivel_profissional_ordinal'] = applicants_clear['informacoes_profissionais.nivel_profissional'].map(nivel_map)

    # Drop the original column
    applicants_clear = applicants_clear.drop(columns=['informacoes_profissionais.nivel_profissional'])

    print("Column 'informacoes_profissionais.nivel_profissional' mapped to 'nivel_profissional_ordinal' and original column dropped.")
    # Optional: Display value counts for the new column
    # print("\nValue counts for 'nivel_profissional_ordinal':")
    # display(applicants_clear['nivel_profissional_ordinal'].value_counts(dropna=False))
else:
    print("DataFrame 'applicants_clear' or the column 'informacoes_profissionais.nivel_profissional' not found.")

Column 'informacoes_profissionais.nivel_profissional' mapped to 'nivel_profissional_ordinal' and original column dropped.


In [ ]:
nivel_academico_map = {
    'Ensino Fundamental Completo': 1,
    'Ensino Médio Incompleto': 2,
    'Ensino Médio Completo': 3,
    'Ensino Técnico Incompleto': 4,
    'Ensino Técnico Cursando': 5,
    'Ensino Técnico Completo': 6,
    'Ensino Superior Incompleto': 7,
    'Ensino Superior Cursando': 8,
    'Ensino Superior Completo': 9,
    'Pós Graduação Incompleto': 10,
    'Pós Graduação Cursando': 11,
    'Pós Graduação Completo': 12,
    'Mestrado Cursando': 13,
    'Mestrado Completo': 14,
    'Doutorado Cursando': 15,
    'Doutorado Completo': 16
}

if 'applicants_clear' in locals() and 'formacao_e_idiomas.nivel_academico' in applicants_clear.columns:
    applicants_clear['nivel_academico_ordinal'] = applicants_clear['formacao_e_idiomas.nivel_academico'].map(nivel_academico_map)
    # Drop the original column
    applicants_clear = applicants_clear.drop(columns=['formacao_e_idiomas.nivel_academico'])
    print("Column 'formacao_e_idiomas.nivel_academico' mapped to 'nivel_academico_ordinal' and original column dropped.")
    # Optional: Display value counts for the new column
    # print("\nValue counts for 'nivel_academico_ordinal':")
    # display(applicants_clear['nivel_academico_ordinal'].value_counts(dropna=False))
else:
    print("DataFrame 'applicants_clear' or the column 'formacao_e_idiomas.nivel_academico' not found.")


Column 'formacao_e_idiomas.nivel_academico' mapped to 'nivel_academico_ordinal' and original column dropped.


In [ ]:
nivel_ingles_map = {
    'Nenhum': 0,
    'Técnico': 1,
    'Básico': 2,
    'Intermediário': 3,
    'Avançado': 4,
    'Fluente': 5
}

if 'applicants_clear' in locals() and 'formacao_e_idiomas.nivel_ingles' in applicants_clear.columns:
    applicants_clear['nivel_ingles_ordinal'] = applicants_clear['formacao_e_idiomas.nivel_ingles'].map(nivel_ingles_map)
    # Drop the original column
    applicants_clear = applicants_clear.drop(columns=['formacao_e_idiomas.nivel_ingles'])
    print("Column 'formacao_e_idiomas.nivel_ingles' mapped to 'nivel_ingles_ordinal' and original column dropped.")
    # Optional: Display value counts for the new column
    # print("\nValue counts for 'nivel_ingles_ordinal':")
    # display(applicants_clear['nivel_ingles_ordinal'].value_counts(dropna=False))
else:
    print("DataFrame 'applicants_clear' or the column 'formacao_e_idiomas.nivel_ingles' not found.")

    # Substituir valores vazios por 'Nenhum' e padronizar
col = 'formacao_e_idiomas.nivel_espanhol'
if 'applicants_clear' in locals() and col in applicants_clear.columns:
    applicants_clear[col] = applicants_clear[col].fillna('').str.strip()
    applicants_clear[col] = applicants_clear[col].replace('', 'Nenhum')

# Mapeamento ordinal
    nivel_espanhol_map = {
        'Nenhum': 0,
        'Técnico': 1,
        'Básico': 2,
        'Intermediário': 3,
        'Avançado': 4,
        'Fluente': 5
    }

    # Aplicar
    applicants_clear['nivel_espanhol_ordinal'] = applicants_clear[col].map(nivel_espanhol_map)

    # Drop the original column
    applicants_clear = applicants_clear.drop(columns=[col])

    print(f"Column '{col}' mapped to 'nivel_espanhol_ordinal' and original column dropped.")
    # Optional: Display value counts for the new column
    # print(f"\nValue counts for 'nivel_espanhol_ordinal':")
    # display(applicants_clear['nivel_espanhol_ordinal'].value_counts(dropna=False))
else:
    print(f"DataFrame 'applicants_clear' or the column '{col}' not found.")

Column 'formacao_e_idiomas.nivel_ingles' mapped to 'nivel_ingles_ordinal' and original column dropped.
Column 'formacao_e_idiomas.nivel_espanhol' mapped to 'nivel_espanhol_ordinal' and original column dropped.


### Opcional: Salvar CSV

In [ ]:
import os

# Create a directory to store the CSV files if it doesn't exist
output_dir = "temp_files"
os.makedirs(output_dir, exist_ok=True)

# Save the applicants_clear DataFrame as a CSV file
if 'applicants_clear' in locals():
    csv_filename = os.path.join(output_dir, "applicants_clear.csv")
    applicants_clear.to_csv(csv_filename, index=True, sep=';') # index=True to keep the 'id' as index in the CSV
    print(f"DataFrame 'applicants_clear' salvo como '{csv_filename}'.")
else:
    print("DataFrame 'applicants_clear' não encontrado.")

DataFrame 'applicants_clear' salvo como 'temp_files/applicants_clear.csv'.


## Prospects

In [ ]:
# prompt: Na coluna prospects do df flat_prospects existem listas de que contém de 0 a vários dicionários. como gerar um tadaframe com os dados desses dicionários e com o valor da coluna id do df original.

import pandas as pd
# Inicializa uma lista vazia para armazenar os dados dos prospects
all_prospects_data = []

# Itera sobre as linhas do DataFrame original
for index, row in flat_dataframes['flat_prospects'].iterrows():
    prospects_list = row['prospects']
    original_id = row['id'] # Captura o valor da coluna 'id' do DataFrame original

    # Verifica se a lista de prospects não é vazia e se é realmente uma lista
    if isinstance(prospects_list, list) and prospects_list:
        # Itera sobre cada dicionário na lista 'prospects'
        for prospect_dict in prospects_list:
            # Cria um dicionário temporário para o prospect atual
            prospect_data = prospect_dict.copy()
            # Adiciona o 'id' do DataFrame original a este dicionário
            prospect_data['original_id'] = original_id
            # Adiciona o dicionário do prospect (com o 'original_id') à lista principal
            all_prospects_data.append(prospect_data)

# Cria um novo DataFrame a partir da lista de dicionários
prospects_clear = pd.DataFrame(all_prospects_data)

# Exibe as primeiras linhas do novo DataFrame expandido
print("\nDataFrame expandido com dados dos prospects:")
display(prospects_clear.head())

# Opcional: Exibe informações sobre o novo DataFrame
print("\nInformações do DataFrame expandido:")
prospects_clear.info()

# Opcional: Exibe o número de linhas no novo DataFrame
print(f"\nNúmero total de linhas no DataFrame expandido: {len(prospects_clear)}")


DataFrame expandido com dados dos prospects:


,nome,codigo,situacao_candidado,data_candidatura,ultima_atualizacao,comentario,recrutador,original_id
0,José Vieira,25632,Encaminhado ao Requisitante,25-03-2021,25-03-2021,"Encaminhado para - PJ R$ 72,00/hora",Ana Lívia Moreira,4530
1,Srta. Isabela Cavalcante,25529,Encaminhado ao Requisitante,22-03-2021,23-03-2021,"encaminhado para - R$ 6.000,00 – CLT Full , n...",Ana Lívia Moreira,4530
2,Sra. Yasmin Fernandes,25364,Contratado pela Decision,17-03-2021,12-04-2021,Data de Inicio: 12/04/2021,Juliana Cassiano,4531
3,Alexia Barbosa,25360,Encaminhado ao Requisitante,17-03-2021,17-03-2021,,Juliana Cassiano,4531
4,Arthur Almeida,26338,Contratado pela Decision,29-04-2021,18-05-2021,,Stella Vieira,4533



Informações do DataFrame expandido:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53759 entries, 0 to 53758
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   nome                53759 non-null  object
 1   codigo              53759 non-null  object
 2   situacao_candidado  53759 non-null  object
 3   data_candidatura    53759 non-null  object
 4   ultima_atualizacao  53759 non-null  object
 5   comentario          53759 non-null  object
 6   recrutador          53759 non-null  object
 7   original_id         53759 non-null  object
dtypes: object(8)
memory usage: 3.3+ MB

Número total de linhas no DataFrame expandido: 53759


In [ ]:
# Check if the DataFrame exists
if 'prospects_clear' in locals() and isinstance(prospects_clear, pd.DataFrame):
    # Rename the columns
    prospects_clear.rename(columns={'codigo': 'id_applicant', 'original_id': 'id_vaga'}, inplace=True)
    print("Columns 'codigo' and 'original_id' renamed.")
    # Optional: Display the updated columns to verify
    # display(prospects_clear.head())
else:
    print("DataFrame 'prospects_clear' not found or is not a DataFrame.")

Columns 'codigo' and 'original_id' renamed.


In [ ]:
classificacao_candidatos = {
    "Encaminhado ao Requisitante": "positiva",
    "Contratado pela Decision": "positiva",
    "Desistiu": "negativa",
    "Documentação PJ": "positiva",
    "Não Aprovado pelo Cliente": "negativa",
    "Prospect": "neutra",
    "Não Aprovado pelo RH": "negativa",
    "Aprovado": "positiva",
    "Não Aprovado pelo Requisitante": "negativa",
    "Inscrito": "neutra",
    "Entrevista Técnica": "positiva",
    "Em avaliação pelo RH": "positiva",
    "Contratado como Hunting": "positiva",
    "Desistiu da Contratação": "negativa",
    "Entrevista com Cliente": "positiva",
    "Documentação CLT": "positiva",
    "Recusado": "negativa",
    "Documentação Cooperado": "positiva",
    "Sem interesse nesta vaga": "negativa",
    "Encaminhar Proposta": "positiva",
    "Proposta Aceita": "positiva"
}


In [ ]:
if 'prospects_clear' in locals() and 'classificacao_candidatos' in locals():
    # Create the 'classificacao' column using the mapping
    prospects_clear['classificacao'] = prospects_clear['situacao_candidado'].map(classificacao_candidatos).fillna('desconhecida') # Fill NaN with 'desconhecida'

    # Define the columns to drop
    columns_to_drop = [
        'nome',
        'situacao_candidado',
        'data_candidatura',
        'ultima_atualizacao',
        'comentario',
        'recrutador'
    ]

    # Drop the specified columns if they exist in the DataFrame
    columns_to_drop_existing = [col for col in columns_to_drop if col in prospects_clear.columns]
    prospects_clear = prospects_clear.drop(columns=columns_to_drop_existing)

    print("Column 'classificacao' created and specified columns removed from 'prospects_clear'.")
    display(prospects_clear.head())
else:
    print("DataFrame 'prospects_clear' or dictionary 'classificacao_candidatos' not found.")

Column 'classificacao' created and specified columns removed from 'prospects_clear'.


,id_applicant,id_vaga,classificacao
0,25632,4530,positiva
1,25529,4530,positiva
2,25364,4531,positiva
3,25360,4531,positiva
4,26338,4533,positiva


In [ ]:
# Create a directory to store the CSV files if it doesn't exist
output_dir = "temp_files"
os.makedirs(output_dir, exist_ok=True)

# Save the prospects_clear DataFrame as a CSV file
if 'prospects_clear' in locals():
    csv_filename = os.path.join(output_dir, "prospects_clear.csv")
    prospects_clear.to_csv(csv_filename, index=True, sep=';') # index=True to keep the 'id' as index in the CSV
    print(f"DataFrame 'prospects_clear' salvo como '{csv_filename}'.")
else:
    print("DataFrame 'prospects_clear' não encontrado.")

DataFrame 'prospects_clear' salvo como 'temp_files/prospects_clear.csv'.


In [ ]:
prospects_clear.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53759 entries, 0 to 53758
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id_applicant   53759 non-null  object
 1   id_vaga        53759 non-null  object
 2   classificacao  53759 non-null  object
dtypes: object(3)
memory usage: 1.2+ MB


In [ ]:
if 'prospects_clear' in locals():
    print("Unique values and their counts for each column in 'prospects_clear':")
    for column in prospects_clear.columns:
        print(f"\n--- Column: {column} ---")
        # Use value_counts(dropna=False) to include NaN values in the count
        print(prospects_clear[column].value_counts(dropna=False))
else:
    print("DataFrame 'prospects_clear' not found.")

Unique values and their counts for each column in 'prospects_clear':

--- Column: id_applicant ---
id_applicant
833      73
1677     70
12450    54
27660    53
16747    45
         ..
25445     1
25454     1
25983     1
50007     1
50009     1
Name: count, Length: 29405, dtype: int64

--- Column: id_vaga ---
id_vaga
8593     25
8594     25
8595     25
9835     25
9937     25
         ..
12020     1
7349      1
7348      1
7341      1
7380      1
Name: count, Length: 11279, dtype: int64

--- Column: classificacao ---
classificacao
neutra      24001
positiva    20750
negativa     9008
Name: count, dtype: int64


# Modelo

In [ ]:
if 'prospects_clear' in locals() and 'applicants_clear' in locals() and 'vagas_clear' in locals():
    # Merge prospects_clear with applicants_clear on applicant id
    # prospects_clear has 'id_applicant' and applicants_clear has 'id' (as index)
    merged_df = pd.merge(prospects_clear, applicants_clear, left_on='id_applicant', right_index=True, how='left')

    # Merge the result with vagas_clear on vaga id
    # merged_df has 'id_vaga' and vagas_clear has 'id' (as index)
    final_df = pd.merge(merged_df, vagas_clear, left_on='id_vaga', right_index=True, how='left')
    #final_df.drop(columns=['id_applicant', 'id_vaga'], inplace=True)

    print("DataFrames merged successfully.")
    display(final_df.head())
    print("\nInformation about the merged DataFrame:")
    final_df.info()
else:
    print("One or more of the required DataFrames (prospects_clear, applicants_clear, vagas_clear) not found.")

DataFrames merged successfully.


,id_applicant,id_vaga,classificacao,informacoes_pessoais.pcd,informacoes.texto_x,tem_outro_idioma_x,estado_Acre_x,estado_Alagoas_x,estado_Amapá_x,estado_Amazonas_x,...,estado_Santa Catarina_y,estado_Sergipe_y,estado_São Paulo_y,estado_Tocantins_y,precisa_equipamento,nivel_profissional_ordinal_y,nivel_academico_ordinal_y,nivel_ingles_ordinal_y,nivel_espanhol_ordinal_y,tem_outro_idioma_y
0,25632,4530,positiva,0.0,cv pt \ndados pessoais\nestado civil: casado\n...,0.0,False,False,False,False,...,False,False,False,False,0.0,5.0,9.0,0.0,0.0,0.0
1,25529,4530,positiva,0.0,"cv pt solteiro, 47 anos\n\nestrada meringuava,...",0.0,False,False,False,False,...,False,False,False,False,0.0,5.0,9.0,0.0,0.0,0.0
2,25364,4531,positiva,0.0,cv pt \n\nárea de atuação: lider de consultori...,0.0,False,False,False,False,...,False,False,True,False,1.0,7.0,3.0,0.0,0.0,0.0
3,25360,4531,positiva,0.0,cv pt informações pessoais\n estado civil: ca...,0.0,False,False,False,False,...,False,False,True,False,1.0,7.0,3.0,0.0,0.0,0.0
4,26338,4533,positiva,0.0,"cv pt solteiro, brasileiro, 21/06/1987\nhabili...",0.0,False,False,False,False,...,False,False,True,False,0.0,7.0,3.0,1.0,5.0,0.0



Information about the merged DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53759 entries, 0 to 53758
Data columns (total 97 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   id_applicant                           53759 non-null  object 
 1   id_vaga                                53759 non-null  object 
 2   classificacao                          53759 non-null  object 
 3   informacoes_pessoais.pcd               45095 non-null  float64
 4   informacoes.texto_x                    45095 non-null  object 
 5   tem_outro_idioma_x                     45095 non-null  float64
 6   estado_Acre_x                          45095 non-null  object 
 7   estado_Alagoas_x                       45095 non-null  object 
 8   estado_Amapá_x                         45095 non-null  object 
 9   estado_Amazonas_x                      45095 non-null  object 
 10  estado_Bahia_x               

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
from tqdm import trange

def embed_text_columns(
    df: pd.DataFrame,
    text_cols,
    model_name: str = "paraphrase-multilingual-MiniLM-L12-v2",
    batch_size: int = 1024,
    device: str | None = None,
    flatten: bool = False
) -> pd.DataFrame:
    """
    Adiciona embeddings de colunas de texto ao próprio DataFrame.

    Params
    ------
    df : DataFrame original (modificado in place e também retornado)
    text_cols : lista com nomes das colunas de texto
    model_name : modelo Sentence-Transformers
    batch_size : tamanho do lote para inferência
    device : 'cpu', 'cuda', 'cuda:0', etc. (auto se None)
    flatten : se True, cria várias colunas float (col_emb_0...); senão, guarda lista/np.array em uma única coluna

    Return
    ------
    df com novas colunas *_emb (ou *_emb_0, *_emb_1...)
    """
    model = SentenceTransformer(model_name, device=device)

    for col in text_cols:
        if col not in df.columns:
            raise KeyError(f"Coluna '{col}' não existe no DataFrame.")

        # Prepara o texto
        texts = df[col].fillna("").astype(str).tolist()

        # Inferência em lotes
        embs = []
        for i in trange(0, len(texts), batch_size, desc=f"Encoding {col}"):
            batch = texts[i:i+batch_size]
            embs.append(
                model.encode(batch, batch_size=batch_size, convert_to_numpy=True, show_progress_bar=False)
            )
        embs = np.vstack(embs)

        if flatten:
            dim = embs.shape[1]
            new_cols = {f"{col}_emb_{i}": embs[:, i] for i in range(dim)}
            df = df.join(pd.DataFrame(new_cols, index=df.index))
        else:
            df[f"{col}_emb"] = list(embs)

    return df


In [ ]:
import torch

# Check if a GPU is available and set the device accordingly
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Pass the device argument to the function call
final_df = embed_text_columns(final_df,['informacoes.texto_x', 'informacoes.texto_y'], device=device, flatten= True)

Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Encoding informacoes.texto_y: 100%|██████████| 53/53 [02:36<00:00,  2.96s/it]


In [ ]:
final_df = final_df.drop(columns=['informacoes.texto_x', 'informacoes.texto_y'])

In [ ]:
# Create a directory to store the CSV files if it doesn't exist
output_dir = "temp_files"
os.makedirs(output_dir, exist_ok=True)

# Save the final_df DataFrame as a CSV file
if 'final_df' in locals():
    csv_filename = os.path.join(output_dir, "final_df.csv")
    final_df.to_csv(csv_filename, index=True, sep=';') # index=True to keep the 'id' as index in the CSV
    print(f"DataFrame 'final_df' salvo como '{csv_filename}'.")
else:
    print("DataFrame 'final_df' não encontrado.")

DataFrame 'final_df' salvo como 'temp_files/final_df.csv'.


In [ ]:
if 'final_df' in locals():
    # Separate features (X) and target (y)
    # Features are all columns except 'id_applicant', 'id_vaga', and 'classificacao'
    X = final_df.drop(columns=['id_applicant', 'id_vaga', 'classificacao'])
    y = final_df['classificacao']

    print("Features (X) and target (y) separated.")
    print("\nFeatures (X) head:")
    display(X.head())
    print("\nTarget (y) head:")
    display(y.head())
    print("\nFeatures (X) info:")
    display(X.info())
    print("\nTarget (y) info:")
    display(y.info())
else:
    print("DataFrame 'final_df' not found.")

Features (X) and target (y) separated.

Features (X) head:


,informacoes_pessoais.pcd,tem_outro_idioma_x,estado_Acre_x,estado_Alagoas_x,estado_Amapá_x,estado_Amazonas_x,estado_Bahia_x,estado_Ceará_x,estado_Distrito Federal_x,estado_Espírito Santo,...,informacoes.texto_y_emb_374,informacoes.texto_y_emb_375,informacoes.texto_y_emb_376,informacoes.texto_y_emb_377,informacoes.texto_y_emb_378,informacoes.texto_y_emb_379,informacoes.texto_y_emb_380,informacoes.texto_y_emb_381,informacoes.texto_y_emb_382,informacoes.texto_y_emb_383
0,0.0,0.0,False,False,False,False,False,False,False,False,...,0.188745,0.116686,0.254900,-0.300845,-0.037634,0.129007,-0.113273,0.134534,-0.283693,0.079421
1,0.0,0.0,False,False,False,False,False,False,False,False,...,0.188745,0.116686,0.254900,-0.300845,-0.037634,0.129007,-0.113273,0.134534,-0.283693,0.079421
2,0.0,0.0,False,False,False,False,False,False,False,False,...,0.111019,0.133816,-0.053330,-0.099305,-0.034247,0.022992,-0.174796,-0.027442,-0.017516,0.038396
3,0.0,0.0,False,False,False,False,False,False,False,False,...,0.111019,0.133816,-0.053330,-0.099305,-0.034247,0.022992,-0.174796,-0.027442,-0.017516,0.038396
4,0.0,0.0,False,False,False,False,False,False,False,False,...,0.096444,0.228329,0.011265,-0.032502,0.052052,0.179051,0.051747,-0.118267,-0.107086,0.164380



Target (y) head:


,classificacao
0,positiva
1,positiva
2,positiva
3,positiva
4,positiva



Features (X) info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53759 entries, 0 to 53758
Columns: 860 entries, informacoes_pessoais.pcd to informacoes.texto_y_emb_383
dtypes: float32(768), float64(25), object(67)
memory usage: 195.2+ MB


None


Target (y) info:
<class 'pandas.core.series.Series'>
RangeIndex: 53759 entries, 0 to 53758
Series name: classificacao
Non-Null Count  Dtype 
--------------  ----- 
53759 non-null  object
dtypes: object(1)
memory usage: 420.1+ KB


None

In [ ]:
# Inspect data types
print("Data types of features (X):")
display(X.info())

# Handle boolean columns by converting them to integers (0 and 1)
# Check for both 'bool' and 'object' types that represent boolean values after merge
boolean_like_cols = X.select_dtypes(include=['bool', 'object']).columns
for col in boolean_like_cols:
    # Attempt to convert to boolean first, then to int. Handle potential errors.
    try:
        X[col] = X[col].astype(bool).astype(int)
        print(f"Converted column '{col}' to int.")
    except ValueError:
        # If conversion to bool fails, it's likely not a boolean column, skip for now.
        print(f"Column '{col}' could not be converted to boolean/int, skipping for now.")






Data types of features (X):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53759 entries, 0 to 53758
Columns: 860 entries, informacoes_pessoais.pcd to informacoes.texto_y_emb_383
dtypes: float32(768), float64(25), object(67)
memory usage: 195.2+ MB


None

Converted column 'estado_Acre_x' to int.
Converted column 'estado_Alagoas_x' to int.
Converted column 'estado_Amapá_x' to int.
Converted column 'estado_Amazonas_x' to int.
Converted column 'estado_Bahia_x' to int.
Converted column 'estado_Ceará_x' to int.
Converted column 'estado_Distrito Federal_x' to int.
Converted column 'estado_Espírito Santo' to int.
Converted column 'estado_Goiás_x' to int.
Converted column 'estado_Maranhão_x' to int.
Converted column 'estado_Mato Grosso_x' to int.
Converted column 'estado_Mato Grosso do Sul_x' to int.
Converted column 'estado_Minas Gerais_x' to int.
Converted column 'estado_Paraná_x' to int.
Converted column 'estado_Paraíba_x' to int.
Converted column 'estado_Pará_x' to int.
Converted column 'estado_Pernambuco_x' to int.
Converted column 'estado_Piauí_x' to int.
Converted column 'estado_Rio Grande do Norte_x' to int.
Converted column 'estado_Rio Grande do Sul_x' to int.
Converted column 'estado_Rio de Janeiro_x' to int.
Converted column 'estado_

In [ ]:
from sklearn.preprocessing import LabelEncoder

if 'y' in locals():
    # Initialize LabelEncoder
    label_encoder = LabelEncoder()

    # Fit and transform the target variable
    y_encoded = label_encoder.fit_transform(y)

    print("Target variable 'classificacao' encoded to numerical values.")
    print("\nEncoded target variable (y_encoded) head:")
    display(y_encoded[:5])
    print("\nOriginal classes and their encoded values:")
    # Display the mapping of original classes to encoded values
    for original_class, encoded_value in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)):
        print(f"{original_class}: {encoded_value}")
else:
    print("Target variable 'y' not found.")

Target variable 'classificacao' encoded to numerical values.

Encoded target variable (y_encoded) head:


array([2, 2, 2, 2, 2])


Original classes and their encoded values:
negativa: 0
neutra: 1
positiva: 2


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import numpy as np

if 'X' in locals() and 'y_encoded' in locals():
    # Split data into training and testing sets
    # Using a test size of 20% and a random state for reproducibility
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

    print("Data split into training and testing sets.")
    print(f"Training set size: {len(X_train)}")
    print(f"Testing set size: {len(X_test)}")

    # Check for and impute missing values in X_train and X_test
    if X_train.isnull().sum().sum() > 0 or X_test.isnull().sum().sum() > 0:
        print("\nMissing values found in training or testing data. Imputing...")
        # Use SimpleImputer with constant strategy and fill_value -1
        imputer = SimpleImputer(strategy='constant', fill_value=-1)

        # Fit on training data and transform both training and testing data
        X_train = imputer.fit_transform(X_train)
        X_test = imputer.transform(X_test)
        print("Missing values imputed using constant strategy with fill_value -1.")

    else:
        print("\nNo missing values found in training or testing data after split.")

else:
    print("Features (X) or encoded target (y_encoded) not found.")

Data split into training and testing sets.
Training set size: 43007
Testing set size: 10752

Missing values found in training or testing data. Imputing...
Missing values imputed using constant strategy with fill_value -1.


In [ ]:
import pandas as pd

if 'y_encoded' in locals():
    # Count the occurrences of each encoded class
    class_counts = pd.Series(y_encoded).value_counts().sort_index()

    print("Count of instances for each encoded class:")
    display(class_counts)

    # Optional: Display the mapping of encoded values back to original class names
    if 'label_encoder' in locals():
        print("\nMapping of encoded values to original class names:")
        for encoded_value, original_class in zip(label_encoder.transform(label_encoder.classes_), label_encoder.classes_):
             print(f"{encoded_value}: {original_class}")

else:
    print("Encoded target variable (y_encoded) not found.")

Count of instances for each encoded class:


,count
0,9008
1,24001
2,20750



Mapping of encoded values to original class names:
0: negativa
1: neutra
2: positiva


In [ ]:
from imblearn.over_sampling import SMOTE
import numpy as np

if 'X_train' in locals() and 'y_train' in locals():
    print("Applying SMOTE to the training data...")

    # Initialize SMOTE. Set a random state for reproducibility.
    # You can adjust k_neighbors if needed, but the default is often a good starting point.
    smote = SMOTE(random_state=42)

    # Apply SMOTE to the training data
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    print("Oversampling with SMOTE completed.")
    print(f"Original training set shape: {X_train.shape}")
    print(f"Resampled training set shape: {X_train_resampled.shape}")

    # Display the new class distribution in the resampled training set
    print("\nNew class distribution in the resampled training set:")
    unique, counts = np.unique(y_train_resampled, return_counts=True)
    display(dict(zip(unique, counts)))

else:
    print("Training data (X_train, y_train) not found.")

Applying SMOTE to the training data...
Oversampling with SMOTE completed.
Original training set shape: (43007, 860)
Resampled training set shape: (57837, 860)

New class distribution in the resampled training set:


{np.int64(0): np.int64(19279),
 np.int64(1): np.int64(19279),
 np.int64(2): np.int64(19279)}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, train_test_split
import xgboost as xgb
import warnings
import pandas as pd
from xgboost import XGBClassifier # Import XGBClassifier


# Suppress the specific warning about use_label_encoder
warnings.filterwarnings("ignore", category=UserWarning, module="xgboost")


if 'X_train_resampled' in locals() and 'y_train_resampled' in locals():
    print("Starting RandomizedSearchCV without Early Stopping during cross-validation...")

    # Separate a hold‑out set from the resampled training data for early stopping
    # This set will not be used during the cross-validation of RandomizedSearchCV
    # but can be used to train the final model with early stopping after the search.
    X_tr, X_val, y_tr, y_val = train_test_split(
        X_train_resampled, y_train_resampled,
        test_size=0.1, # Use 10% of the resampled data for validation
        stratify=y_train_resampled, # Keep class distribution similar
        random_state=42
    )

    # Define the parameter distribution to search (for RandomizedSearchCV)
    param_dist = {
        'learning_rate': [0.05, 0.1, 0.5],
        'max_depth': [6, 8, 10],
        'n_estimators': [200, 300, 500], # Increased range for n_estimators
    }

    # Define fit_params - Remove early stopping parameters from here
    # as they are causing issues when passed via RandomizedSearchCV.fit()
    fit_params = {}


    # Helper function to create and run the RandomizedSearch
    def run_random_search(tree_method):
        model = XGBClassifier(
            objective='multi:softmax',
            eval_metric='mlogloss',
            use_label_encoder=False,
            random_state=42,
            tree_method=tree_method,
            n_jobs=-1 # Use all available cores for individual model training
        )
        search = RandomizedSearchCV(
            estimator=model,
            param_distributions=param_dist,
            n_iter=50,            # number of random combinations to try
            cv=3,                 # 3-fold cross-validation
            scoring='f1_weighted', # Use f1_weighted because of potential imbalance in validation set (even after SMOTE)
            n_jobs=-1,            # Use all available cores for RandomizedSearchCV (parallelizes folds and iterations)
            verbose=2,
            random_state=42,
            return_train_score=True # Optional: to inspect for overfitting
        )
        # Call fit without passing fit_params for early stopping to avoid TypeError
        search.fit(X_tr, y_tr)
        return search

    # Try GPU first
    try:
        # Check if XGBoost is compiled with GPU support by trying to instantiate and fit a small model
        temp_model = XGBClassifier(tree_method='gpu_hist')
        temp_model.fit(X_tr[:10], y_tr[:10]) # Attempt a quick fit to verify
        print("GPU support detected. Running with GPU.")
        random_search = run_random_search(tree_method='gpu_hist')
        print("RandomizedSearchCV with GPU completed successfully.")
    except Exception as e: # Catching a general Exception
        print(f"GPU failure or lack of support: {e}")
        print("Falling back to CPU...")
        random_search = run_random_search(tree_method='hist')
        print("RandomizedSearchCV with CPU completed successfully.")

    # Display results
    results = pd.DataFrame(random_search.cv_results_)
    # Select relevant columns for display
    display_cols = ['params', 'mean_test_score', 'std_test_score', 'rank_test_score', 'mean_train_score', 'std_train_score']
    display(results[display_cols])


    print(f"\nBest parameters found: {random_search.best_params_}")
    print(f"Best cross-validation F1-weighted score: {random_search.best_score_:.4f}")

    # The best fitted model is available in random_search.best_estimator_
    best_model = random_search.best_estimator_

else:
    print("Resampled training data (X_train_resampled, y_train_resampled) not found.")

Starting RandomizedSearchCV without Early Stopping during cross-validation...
GPU support detected. Running with GPU.
Fitting 3 folds for each of 27 candidates, totalling 81 fits


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 27 is smaller than n_iter=50. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV with GPU completed successfully.


,params,mean_test_score,std_test_score,rank_test_score,mean_train_score,std_train_score
0,"{'n_estimators': 200, 'max_depth': 6, 'learnin...",0.634939,0.002860,27,0.826649,0.000525
1,"{'n_estimators': 300, 'max_depth': 6, 'learnin...",0.649015,0.004056,26,0.875246,0.001216
2,"{'n_estimators': 500, 'max_depth': 6, 'learnin...",0.666299,0.003923,24,0.928633,0.002138
3,"{'n_estimators': 200, 'max_depth': 8, 'learnin...",0.667619,0.003831,23,0.943445,0.000778
4,"{'n_estimators': 300, 'max_depth': 8, 'learnin...",0.677875,0.003598,20,0.960953,0.001290
5,"{'n_estimators': 500, 'max_depth': 8, 'learnin...",0.688457,0.001813,9,0.970168,0.000627
6,"{'n_estimators': 200, 'max_depth': 10, 'learni...",0.683452,0.002211,16,0.969188,0.000970
7,"{'n_estimators': 300, 'max_depth': 10, 'learni...",0.690111,0.001985,7,0.971426,0.000521
8,"{'n_estimators': 500, 'max_depth': 10, 'learni...",0.696812,0.003345,2,0.972030,0.000439
9,"{'n_estimators': 200, 'max_depth': 6, 'learnin...",0.656599,0.002801,25,0.906781,0.001612



Best parameters found: {'n_estimators': 500, 'max_depth': 10, 'learning_rate': 0.1}
Best cross-validation F1-weighted score: 0.6978


In [ ]:
# 1) Salvar o modelo em disco
import joblib

# supondo que seu modelo esteja na variável best_model
joblib.dump(best_model, 'best_model.pkl')
print("Modelo salvo como best_model.pkl")

# 2) Forçar o download para o seu computador local
from google.colab import files
files.download('best_model.pkl')

Modelo salvo como best_model.pkl


/usr/lib/python3.11/pickle.py:578: UserWarning: [00:18:30] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  rv = reduce(self.proto)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

FileNotFoundError: Cannot find file: final_df.csv

In [ ]:
import os

In [ ]:
import joblib

# Load the model from the file
if os.path.exists('best_model.pkl'):
    model = joblib.load('best_model.pkl')
    print("Model loaded successfully from 'best_model.pkl'.")
else:
    print("Error: Model file 'best_model.pkl' not found.")
    model = None # Or handle the error as appropriate

Model loaded successfully from 'best_model.pkl'.


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

if 'model' in locals() and 'X_test' in locals() and 'y_test' in locals() and 'label_encoder' in locals():
    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Evaluate the model
    print("Model Evaluation on Test Set:")

    # Accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"\nAccuracy: {accuracy:.4f}")

    # Classification Report (includes precision, recall, f1-score for each class)
    # Use target_names to show original class names in the report
    target_names = label_encoder.classes_
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, target_names=target_names))

    # Confusion Matrix
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

else:
    print("Trained model, test data, or label_encoder not found.")

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:2676: UserWarning: [11:53:27] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  if len(data.shape) != 1 and self.num_features() != data.shape[1]:
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:729: UserWarning: [11:53:28] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)


Model Evaluation on Test Set:

Accuracy: 0.5988

Classification Report:
              precision    recall  f1-score   support

    negativa       0.45      0.35      0.39      1806
      neutra       0.63      0.70      0.66      4722
    positiva       0.61      0.59      0.60      4224

    accuracy                           0.60     10752
   macro avg       0.56      0.55      0.55     10752
weighted avg       0.59      0.60      0.59     10752


Confusion Matrix:
[[ 629  625  552]
 [ 346 3296 1080]
 [ 418 1293 2513]]
